<a href="https://colab.research.google.com/github/tnfru/colab_research/blob/master/CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR100
import torchvision.models
from torchvision import transforms as T
from torch.utils.data import DataLoader
from torchsummary import summary
import time
import wandb

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device:', DEVICE)
torch.backends.cudnn.benchmark = True

using device: cuda


In [3]:
BATCH_SIZE = 128
TRAIN_MEANS = np.array([0.50707516, 0.48654887, 0.44091784])
TRAIN_STDS = np.array([0.26733429, 0.25643846, 0.27615047])
TEST_MEANS = np.array([0.50879641, 0.48739301, 0.44194221])
TEST_STDS = np.array([0.26825157, 0.25736374, 0.27709577])

In [4]:
tr_train = T.Compose([T.ToTensor(),
                      T.Normalize(TRAIN_MEANS, TRAIN_STDS)])

tr_test = T.Compose([T.ToTensor(),
                     T.Normalize(TEST_MEANS, TEST_STDS)])

train = CIFAR100("./data", train=True, download=True, transform=tr_train)
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

test = CIFAR100("./data", train=False, download=True, transform=tr_test)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class IdentityModule(nn.Module):
  def __init__(self, num_channel):
    super(IdentityModule, self).__init__()

    self.hidden1 = nn.Sequential(
        nn.Conv2d(num_channel, num_channel, 3, padding=1, bias=False),
        nn.BatchNorm2d(num_channel),
        nn.ReLU(inplace=True),
        nn.Conv2d(num_channel, num_channel, 3, padding=1, bias=False),
        nn.BatchNorm2d(num_channel)
    )

    self.hidden2 = nn.Sequential(
        nn.Conv2d(num_channel, num_channel, 3, padding=1, bias=False),
        nn.BatchNorm2d(num_channel),
        nn.ReLU(inplace=True),
        nn.Conv2d(num_channel, num_channel, 3, padding=1, bias=False),
        nn.BatchNorm2d(num_channel)
    )
  
    self.hidden3 = nn.Sequential(
        nn.Conv2d(num_channel, num_channel, 3, padding=1, bias=False),
        nn.BatchNorm2d(num_channel),
        nn.ReLU(inplace=True),
        nn.Conv2d(num_channel, num_channel, 3, padding=1, bias=False),
        nn.BatchNorm2d(num_channel)
    )

    self.relu = nn.ReLU(inplace=True)

  def forward(self, x):
    x = self.hidden1(x) + x
    x = self.relu(x)
    x = self.hidden2(x) + x
    x = self.relu(x)
    x = self.hidden3(x) + x
    x = self.relu(x)

    return x

In [6]:
class ConvModule(nn.Module):
  def __init__(self, in_channel, out_channel, stride=2):
    super(ConvModule, self).__init__()

    self.hidden1 = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, 3, padding=1, stride=stride, bias=False),
        nn.BatchNorm2d(out_channel),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channel, out_channel, 3, padding=1, bias=False),
        nn.BatchNorm2d(out_channel)
    )
    self.hidden2 = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, 1, stride=stride, bias=False),
        nn.BatchNorm2d(out_channel)
    )

    self.relu = nn.ReLU(inplace=True)

  def forward(self, x):
    x1 = self.hidden1(x)
    x2 = self.hidden2(x)
    x = x1 + x2
    x = self.relu(x)

    return x


In [7]:
class ResNet(nn.Module):
  def __init__(self):
    super(ResNet, self).__init__()
    self.inp = nn.Sequential(
        nn.Conv2d(3, 64, 3, padding=1, bias=False),
        nn.BatchNorm2d(64)
    )

    self.hidden1 = nn.Sequential(
        IdentityModule(64),
    )

    self.hidden2 = nn.Sequential(
        ConvModule(64, 128),
        IdentityModule(128),
        nn.Dropout(0.15)
    )

    self.hidden3 = nn.Sequential(
        ConvModule(128, 256),
        IdentityModule(256),
        nn.Dropout(0.2)
    )

    self.out = nn.Sequential(
        nn.AvgPool2d(kernel_size=8),
        nn.Flatten(),
        nn.Linear(256, 100),
        nn.ReLU(),
        nn.Linear(100, 100),
        nn.Softmax(dim=1)
    )

  def forward(self, x):
    x = self.inp(x)
    x = self.hidden1(x)
    x = self.hidden2(x)
    x = self.hidden3(x)
    x = self.out(x)

    return x

In [8]:
def test_accuracy(model, history):
  model.eval()
  with torch.no_grad():
      correct = 0
      total = 0

      for data, target in test_loader:
          images = data.to(DEVICE)
          labels = target.to(DEVICE)
          outputs = model(images)

          _, predicted = torch.max(outputs, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

          accuracy = 100 * correct / total
          history.append(accuracy)

  model.train()

  return history

In [9]:
def train_model(model, opt, scheduler, num_epochs, history=[], verbose=True):
  criterion = nn.CrossEntropyLoss()

  for epoch in range(num_epochs):
    correct = 0
    total = 0
    start_time = time.time()

    for i, (images, labels) in enumerate(train_loader):
      # Write tensor to GPU if available
      images, labels = images.to(DEVICE), labels.to(DEVICE)

      # Clear Gradient
      opt.zero_grad()

      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)

      # Backward pass
      loss.backward()
      opt.step()

    history = test_accuracy(model, history)

    if verbose:
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

      train_accuracy = 100 * correct / total
      print('EPOCH: {} LR: {} Time: {:.4} sec Train Accuracy: {} % Test Accuracy: {} %'.format(epoch, 
            scheduler.get_last_lr(), time.time() - start_time, train_accuracy, history[-1]))
      
      wandb.log({
          "epoch": epoch,
          "lr": scheduler.get_last_lr(),
          "time": time.time() - start_time,
          "train_acc": train_accuracy,
          "test_acc": history[-1]
      })
    
    scheduler.step()

  return history

In [12]:
config = wandb.config
config.lr = 1e-3
config.epochs = 150
config.log_interval = 2
config.batch_size = BATCH_SIZE

wandb: Currently logged in as: tnfru (use `wandb login --relogin` to force relogin)


In [13]:
model = ResNet()
#model = torchvision.models.resnet50(num_classes=100)
model.to(DEVICE)
opt = optim.Adam(model.parameters(), lr=config.lr)
scheduler = optim.lr_scheduler.MultiStepLR(opt, milestones=[50, 100], gamma=0.1)
history = []

In [14]:
wandb.init(project="cifar-100", name="test_run_resnet_no_wide")
wandb.watch(model, log="all")

history = train_model(model, opt, scheduler, num_epochs=config.epochs, history=history)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


EPOCH: 0 LR: [0.001] Time: 54.42 sec Train Accuracy: 3.75 % Test Accuracy: 3.69 %
EPOCH: 1 LR: [0.001] Time: 54.31 sec Train Accuracy: 5.0 % Test Accuracy: 3.83 %
EPOCH: 2 LR: [0.001] Time: 54.55 sec Train Accuracy: 7.5 % Test Accuracy: 5.37 %
EPOCH: 3 LR: [0.001] Time: 54.23 sec Train Accuracy: 7.5 % Test Accuracy: 5.63 %
EPOCH: 4 LR: [0.001] Time: 54.5 sec Train Accuracy: 7.5 % Test Accuracy: 6.9 %
EPOCH: 5 LR: [0.001] Time: 54.41 sec Train Accuracy: 8.75 % Test Accuracy: 6.97 %
EPOCH: 6 LR: [0.001] Time: 54.49 sec Train Accuracy: 7.5 % Test Accuracy: 7.16 %
EPOCH: 7 LR: [0.001] Time: 54.34 sec Train Accuracy: 8.75 % Test Accuracy: 7.48 %
EPOCH: 8 LR: [0.001] Time: 54.61 sec Train Accuracy: 8.75 % Test Accuracy: 8.06 %
EPOCH: 9 LR: [0.001] Time: 54.39 sec Train Accuracy: 10.0 % Test Accuracy: 7.99 %
EPOCH: 10 LR: [0.001] Time: 54.64 sec Train Accuracy: 10.0 % Test Accuracy: 8.75 %
EPOCH: 11 LR: [0.001] Time: 54.27 sec Train Accuracy: 10.0 % Test Accuracy: 8.03 %
EPOCH: 12 LR: [0.001]

KeyboardInterrupt: ignored